In [3]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import numpy as np
import ast
from util import check_user_exists, lookup_product_name
import model_functions as mf

In [4]:
users_final = pd.read_csv('https://raw.githubusercontent.com/ardahk/amex/refs/heads/main/final/users_final_data.csv')
products_final= pd.read_csv('https://raw.githubusercontent.com/ardahk/amex/refs/heads/main/final/products_final_data.csv')
original_products = pd.read_csv('https://raw.githubusercontent.com/ardahk/amex/refs/heads/main/data/products.csv')

## Building baseline 2 tower model

### The first issue is that for each training batch, we need to have the same amount of user-item pairs as input. This means we need to use some sort of sampling for each batch in order to make sure they're both the same size.

In [5]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Concatenate, Dot, BatchNormalization
from tensorflow.keras.models import Model

2024-12-01 23:22:58.197812: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-01 23:22:58.200952: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-01 23:22:58.207540: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1733095378.218037    5663 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733095378.221186    5663 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-01 23:22:58.233940: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

In [6]:
user_input = Input(shape=(16,), name='user_input')
item_input = Input(shape=(30,), name='item_input')

In [7]:
#Changed from baseline
user_tower = Dense(128, activation='relu')(user_input)
user_tower = BatchNormalization()(user_tower)

2024-12-01 23:23:00.005696: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [8]:
item_tower = Dense(128, activation='relu')(item_input)
item_tower = BatchNormalization()(item_tower)

In [9]:
dot_product = Dot(axes=1)([user_tower, item_tower])

In [10]:
model = Model(inputs=[user_input, item_input], outputs=dot_product)

In [11]:
model.compile(optimizer='adam', loss='mse')

In [12]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ user_input          │ (None, 16)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ item_input          │ (None, 30)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 128)       │      2,176 │ user_input[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 128)       │      3,968 │ item_input[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 128)       │        512 │ dense[0][0]       │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128)       │        512 │ dense_1[0][0]     │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dot (Dot)           │ (None, 1)         │          0 │ batch_normalizat… │
│                     │                   │            │ batch_normalizat… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 7,168 (28.00 KB)

 Trainable params: 6,656 (26.00 KB)

 Non-trainable params: 512 (2.00 KB)

In [14]:
# Parameters
batch_size = 500
num_epochs = 20

acc, precision = mf.create_labels_and_train(users_final, products_final, model, batch_size, num_epochs)
print(acc)
print(precision)

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
0.56
0.515625


The above code splits the data into training and testing data, and trains the model.

## Generating Recommendations

In [12]:
original_products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29120 entries, 0 to 29119
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   id                      29120 non-null  int64  
 1   cost                    29120 non-null  float64
 2   category                29120 non-null  object 
 3   name                    29118 non-null  object 
 4   brand                   29096 non-null  object 
 5   retail_price            29120 non-null  float64
 6   department              29120 non-null  object 
 7   sku                     29120 non-null  object 
 8   distribution_center_id  29120 non-null  int64  
dtypes: float64(2), int64(2), object(5)
memory usage: 2.0+ MB


In [13]:
recs = mf.generate_recommendations(users_final, products_final, model, top_n=10)
recs
lookup_product_name(recs, original_products)

Generating recommendations for user ID: 17148...
User data repeated shape: (13800, 16)
Product data shape: (13800, 30)
432/432 ━━━━━━━━━━━━━━━━━━━━ 0s 843us/step
ID = 14202, Name = 11644    GENUINE LEATHER SNAP ON STUDDED WHITE PIANO BE...
Name: name, dtype: object
ID = 13629, Name = 28484    Solid Color Leather Adjustable Skinny Belt with
Name: name, dtype: object
ID = 14298, Name = 16355    Classic Tear Drop Mirror Lens Aviator Sunglasses
Name: name, dtype: object
ID = 14235, Name = 1401    Indestructable Aluminum Aluma Wallet - RED
Name: name, dtype: object
ID = 3049, Name = 21577    Pink Ribbon Breast Cancer Awareness Knee High ...
Name: name, dtype: object
ID = 13606, Name = 11091    Elegant PASHMINA SCARF WRAP SHAWL STOLE
Name: name, dtype: object
ID = 12536, Name = 13235    Individual Bra Extenders
Name: name, dtype: object
ID = 28700, Name = 23027    Wayfarer Style Sunglasses Dark Lens Black Frame
Name: name, dtype: object
ID = 9204, Name = 21579    Pink Ribbon Breast Cancer Aw

### Generating Recommendations for a specific user

In [ ]:
def generate_recommendations_for_user(test_users_df, products_df, model, user_id, top_n=10):
    # Get the data for the specific user
    user_row = test_users_df[test_users_df['user_id'] == user_id]
    
    if user_row.empty:
        raise ValueError(f"User with ID {user_id} not found in the test_users_df.")
    
    print(f"Generating recommendations for user ID: {user_id}...")

    # Prepare user data for the selected user
    user_data = user_row.drop(columns=['product_id', 'user_id']).values

    # Prepare product data (drop 'product_id' and any embeddings)
    product_data = products_df.drop(columns=['product_id', 'flattened_name_embedding', 'flattened_brand_embedding']).values

    # Repeat user data for each product (model requires this structure)
    user_data_repeated = np.repeat(user_data, product_data.shape[0], axis=0)

    print("User data repeated shape:", user_data_repeated.shape)
    print("Product data shape:", product_data.shape)
    # Predict probabilities (model expects a list of two inputs: user data and product data)
    predicted_probabilities = model.predict([user_data_repeated, product_data]).flatten()

    # Sort product recommendations by predicted probabilities
    sorted_indices = np.argsort(predicted_probabilities)
    sorted_products = products_df.iloc[sorted_indices]

    # Get the top N recommendations
    top_recommendations = sorted_products.head(top_n)

    # Return a list of the top N product IDs
    return top_recommendations[['product_id']]['product_id'].tolist()


In [ ]:
check_user_exists(users_final, 84533)

User ID 84533 exists in the DataFrame.


True

In [ ]:
recs = generate_recommendations_for_user(users_final, products_final, model, 84533, top_n=10)
lookup_product_name(recs, original_products)

Generating recommendations for user ID: 84533...
User data repeated shape: (55200, 16)
Product data shape: (13800, 30)


ValueError: Data cardinality is ambiguous. Make sure all arrays contain the same number of samples.'x' sizes: 55200, 13800
